In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time

def record_object_es2(uuid_list, con_key, schema_name, store_frame='raw', add_pc_wfr=False, store={}, item_uuids=[]):
    """starting from a single uuid, tracks all linked items,
    keeps a list of uuids, and dictionary of items for each schema in the given store_frame"""
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    if add_pc_wfr:
        add_from_embedded = {'file_fastq': ['workflow_run_inputs', 'workflow_run_outputs'],
                             'file_processed': ['workflow_run_inputs', 'workflow_run_outputs']
                             }
    else:
        add_from_embedded = {}

    #find schema name, store as obj_key, create empty list if missing in store
    ES_items = ff_utils.get_es_metadata(uuid_list, key=con_key)

    for ES_item in ES_items:
        uuids_to_check = []
        uuid = ES_item['uuid']
        object_resp = ES_item['object']
        obj_type = object_resp['@type'][0]
        obj_key = schema_name[obj_type]
        if obj_key not in store:
            store[obj_key] = []
        raw_resp = ES_item['properties']
        raw_resp['uuid'] = uuid
        # add raw frame to store and uuid to list
        if uuid not in item_uuids:
            if store_frame == 'object':
                store[obj_key].append(object_resp)
            else:
                store[obj_key].append(raw_resp)
            item_uuids.append(uuid)
        # this case should not happen actually
        else:
            print('Problem encountered - skipped - check')
            return store, item_uuids

        #get linked items from es
        for key in ES_item['links']:
            uuids_to_check.extend(ES_item['links'][key])

        # check if any field from the embedded frame is required
        add_fields = add_from_embedded.get(obj_key)
        if add_fields:
            emb_resp = ES_item['embedded']
            for a_field in add_fields:
                field_val = emb_resp.get(a_field)
                if field_val:
                    #turn it into string
                    field_val = str(field_val)
                    # check if any of embedded uuids is in the field value
                    for a_uuid in ES_item['embedded_uuids']:
                        if a_uuid in field_val:
                            uuids_to_check.append(a_uuid)
        # get uniques
        uuids_to_check = list(set(uuids_to_check))
        uuids_to_pass = []
        for an_uuid in uuids_to_check:
            if an_uuid not in item_uuids:
                uuids_to_pass.append(an_uuid)
        if uuids_to_pass:
            print(len(uuids_to_pass))
            store, item_uuids = record_object_es2(uuids_to_pass, con_key, schema_name, store_frame, add_pc_wfr, store, item_uuids)
    return store, item_uuids

In [4]:
time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 
set_to_release = ['69b42ba9-4333-48d7-bb52-75e33572e777', '0365d3e1-23b2-4dc6-ba7c-23a9555ca106', 'd00eb4c2-e2e7-4c2d-aea8-52647cecc2d7', 'd7d3a9d3-0b4e-4a5e-bb73-2414d7e70dee', '53356f5b-5193-4cb7-82af-7b5830536988', '7fd0b68a-06a1-45e7-8f0d-3994062558bb', '01b77208-e4e7-4da3-93bb-54ef38e94c22', '75fcb7cb-3860-4b89-be0c-90045ce96db6', 'fdb789cc-8160-42f8-b105-c3d69417877c', '16fce128-e56e-40dd-852f-3b8eecc818ea', 'cb5a9dc3-3b93-4ccc-aa7b-53215c801e18', '3442ff9a-4f3a-477c-921e-52e5599000a2', '8a7937cd-fa94-48c7-94ae-51691495a282', '13b99a06-520e-4444-ba09-576cd1f72671', '6812d6ac-a98a-448e-a16b-4b9c9ca5d8c8', '47b6ef45-377f-4c66-ae4f-84732fc2b71d', 'daad58e7-4393-4568-8c60-4a8f028aaef4', '967b8fde-96e9-4da6-b1d3-87736b833317', '891be5a2-c15a-4376-b8e0-3ecfb119f71a', '287638c3-8a38-418f-af04-b16349b500b7', '1d3d4f8f-ce02-455c-b511-11968abb3ba7', '1756771b-9476-4f19-91dc-a4df60713e65', 'a137500c-b54e-4076-9a6b-3628281d701e', 'c0df1347-506b-4215-89b5-388f113f1102', '01e0dc4c-8a1f-436a-883a-3c4d9cf54584', '806be02f-5fa4-45e7-b315-6a02cbce340f', '50d81c86-9cb3-4c79-a03c-93fa7e7a5bf0', '67496616-e288-4f55-b6be-dc047681b5da', '833b51d8-32a1-43e1-9fb7-22b352b81322', '19e64440-efb5-416b-bfe6-a3441cda1494', '6d7e7404-9ae8-4418-8a03-c221f56f3e11', 'a689a1e7-306e-4bd9-b821-1c8e771c24e9', '014c48c0-a6de-485d-8ba8-5a88f20cc3ba', 'a234a856-188b-4ab5-897a-47786fd00e60', '67bb8663-c830-4283-92e5-2f4aada3b18e', '36a04e7d-a102-46ac-9a39-46255107cf62', 'dc6b05b0-7375-4e53-9089-9b9e648a736f', '49757b2a-6305-40ef-b619-900bce77c7f4', 'ecb8b23e-08ba-4856-87c8-5d03c51b46a9', '132c3356-f4e9-486a-b616-dc068a1c5ecb', 'f102d8ab-4d65-4b50-bc51-48f0ec575b31', '30ce209c-aa3f-4d8d-b685-ca109d3b7256', 'e7adc531-c594-4f3e-a432-252dd5b5748d', 'f81ebf64-2b1a-4214-959c-cbb8deee34e2', '59b0a231-36ca-4f91-9582-70ca3ba011bc', '5a0a05e1-2c85-431c-8c56-2e63a78f92b3', 'fe30590a-6cd0-4059-9e0f-972088e6ebb8', 'd9be087a-97a0-4b5b-8ef4-284c35ebde4d', '8e6e3e89-12d1-4ae9-bb22-1e23cfbe1beb', '0c63a6cf-98f0-4792-a7f1-993093bb12aa', '86ea6352-be54-41b7-80af-d057a0a3069d', '1cf3e3cc-ed33-40f5-a050-904cd45fa774', 'd3092299-0e5f-4547-a992-c4cdd0a8b3b0', 'ae543a12-d1f9-4691-add7-e458fbfee682', 'a058f8d5-553b-4de6-8873-82d44bcc85c4', '8e76b675-d6a1-40d1-a6ba-ca0b4dea4124', '5fd6a6bf-8b7c-4137-8772-94128e8b934a', 'd6ad785e-900b-435e-ad61-d04c34569505', 'bbcd6f43-94d7-404d-a00f-76a73daadfd2', '09669e98-dc9c-432a-8ef0-25d15fe39650', 'c41d6cdb-de46-439c-9823-10efd30f11e5', 'bd13580d-3f70-455e-98ff-bc4c118bdeef', '93b532d5-35a7-4df8-b2b9-97082de7d22d']

store, uuidsrecord_object_es2(set_to_release, , my_auth, schema_name, store_frame='raw', add_pc_wfr=False)


time2 = time.time()
print((time2-time1)/60)

2
2

15
15

28
28

41
41

54
54

67
67

80
80

93
93

106
106

119
119

50
0.0474394162495931


In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time
time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 

set_to_release = ['83e74398-ac79-4372-91d3-250e7390d636']
store, uuids = record_object_es2(set_to_release, my_auth, schema_name, add_pc_wfr=True, store={}, item_uuids=[])
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1)/60)

63
2
2
1
1
1
Problem encountered - skipped - check
Problem encountered - skipped - check
7
6
3
1
1
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
5
4
1
4
2
1
4
2
2
2
1
6
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
1
1
1
1
2
1
1
1
5
2
1
2
1
1
1
1
1
1
1
5
3
1
4
4
2
1
Problem encountered - skipped - check
1
Problem encountered - skipped - check
1
1
1
1
1
1
1
5
1
1
5
1
1
5
2
2
1
1
1
1
1
1
1
5
2
1
5
1
147
147
0.5848649501800537


In [ ]:
# Release the items
action = False
change_status = 'released'

counter = 0
for a_type in store:
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print('deleted data', a_type ,raw_data['uuid'])
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print(a_type, raw_data['status'], raw_data['uuid'])
            patch_data = {'status': "released"}
            if action:
                ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
        else:
            print('ALREADY RELEASED/TO PROJECT', a_type, raw_data['status'], raw_data['uuid'])
            pass
            
print(counter)